In [29]:
# мой рекомендательный алгоритм начинается с блока 9
import numpy as np
import pandas as pd
import math


In [30]:
KNN = 4
MY_USER = 22
NO_RATE = -1
def readAndFormat(file):
    table = pd.read_csv(file, delimiter=', ')
    table = table.drop(columns=[table.columns[0]])
    return table
data = readAndFormat('data.csv')

In [31]:
def getSimDict(data, relativeToUser):
    sim = dict(zip(list(data.index.values) , [0 for i in range(len(data))]))
    del sim[relativeToUser]
    for user in sim.keys():
        set = data.iloc[[user, relativeToUser]]
        set = set.transpose()
        set = set[(set[user] != NO_RATE) & (set[relativeToUser] != NO_RATE )]
        sumuv = 0
        sumuu = 0
        sumvv = 0
        for index, row in set.iterrows():
            sumuv +=row[user] * row[relativeToUser]
            sumuu +=row[user] * row[user]
            sumvv +=row[relativeToUser] * row[relativeToUser]
        sim[user] = round(sumuv / (math.sqrt(sumuu) * math.sqrt(sumvv)), 3)
    return sim

In [32]:
def headSortedDict(dictionary, n):
    return dict(sorted(list(dictionary.items()), key=lambda x: x[1], reverse=True)[:n])


In [33]:

def meanForDataFrame(data):
    def tupleToMean(x):
        l = np.array(list(filter(lambda x: x != NO_RATE, x)))
        user = l[0]
        l = np.delete(l, [0])
        return [user, l.mean()]
    return dict(list(map(tupleToMean, data.to_records())))



In [34]:
# неоцененные фильмы
def getUnratedMovies(data, user):
    return list(data.loc[user][data.loc[user] == NO_RATE].keys())


In [35]:
UNRATED = getUnratedMovies(data, MY_USER)
UNRATED

['Movie 3',
 'Movie 6',
 'Movie 7',
 'Movie 9',
 'Movie 21',
 'Movie 22',
 'Movie 26']

In [36]:
SIM = getSimDict(data, MY_USER)
SIM

{0: 0.831,
 1: 0.798,
 2: 0.825,
 3: 0.903,
 4: 0.885,
 5: 0.836,
 6: 0.865,
 7: 0.785,
 8: 0.721,
 9: 0.868,
 10: 0.738,
 11: 0.869,
 12: 0.844,
 13: 0.719,
 14: 0.854,
 15: 0.839,
 16: 0.815,
 17: 0.817,
 18: 0.772,
 19: 0.888,
 20: 0.863,
 21: 0.845,
 23: 0.824,
 24: 0.802,
 25: 0.763,
 26: 0.916,
 27: 0.873,
 28: 0.807,
 29: 0.779,
 30: 0.817,
 31: 0.811,
 32: 0.801,
 33: 0.787,
 34: 0.862,
 35: 0.883,
 36: 0.725,
 37: 0.859,
 38: 0.753,
 39: 0.794}

In [37]:
CLOSEST_SIM = headSortedDict(SIM, KNN)
CLOSEST_SIM

{26: 0.916, 3: 0.903, 19: 0.888, 4: 0.885}

In [38]:
MEANS = meanForDataFrame(data.iloc[[*list(CLOSEST_SIM.keys()), MY_USER]])
MEANS

{26: 3.3333333333333335,
 3: 3.142857142857143,
 19: 3.1818181818181817,
 4: 3.3076923076923075,
 22: 3.391304347826087}

In [39]:
newRates = {}
for film in UNRATED:
    sumt = 0
    sumb = 0
    for user in CLOSEST_SIM.keys():
        rate = data.loc[user, film]
        if rate == NO_RATE:
            continue 
        sumb += SIM[user]
        sumt += SIM[user] * (data.loc[user, film] - MEANS[user])
    newRates[film] = round(MEANS[MY_USER] + sumt/sumb, 3)

print(newRates)

{'Movie 3': 2.651, 'Movie 6': 3.474, 'Movie 7': 2.907, 'Movie 9': 3.144, 'Movie 21': 4.084, 'Movie 22': 3.144, 'Movie 26': 3.898}


In [40]:
dataDays = readAndFormat('context_day.csv')
dataPlaces =readAndFormat('context_place.csv')
DAYS = ['Sat', 'Sun']
PLACES = ['h']

In [41]:
# Возьмем 4 людей с наиболее близкими вкусами

In [42]:
# посчитаем оценку фильмов, которые не смотрел пользователь как выше, но добавим контекстый вес:
# для учета оценки, хотя бы 1 контекст должен совпасть 
# если совпали оба контекста, учтем оценку дополнительный раз, тем самым, увеличим ее все
newRatesWithContext = {}
for film in UNRATED:
    sumt = 0
    sumb = 0
    for user in CLOSEST_SIM.keys():
        rate = data.loc[user, film]
        if rate == NO_RATE:
            continue 
        if dataDays.loc[user, film] in DAYS:
            sumb += SIM[user]
            sumt += SIM[user] * (data.loc[user, film] - MEANS[user])
        if dataPlaces.loc[user, film] in PLACES:
            sumb += SIM[user]
            sumt += SIM[user] * (data.loc[user, film] - MEANS[user])
    newRatesWithContext[film] = round(MEANS[MY_USER] + (sumt/sumb if sumb !=0 else 0) , 3)

print(newRatesWithContext)

{'Movie 3': 2.223, 'Movie 6': 2.523, 'Movie 7': 2.855, 'Movie 9': 3.058, 'Movie 21': 3.391, 'Movie 22': 2.229, 'Movie 26': 4.145}


In [43]:
# выбирем фильм с наибольшей оценкой
BEST_SUGGESTION = headSortedDict(newRatesWithContext, 1)
BEST_SUGGESTION

{'Movie 26': 4.145}

In [44]:
# формируем результат и пишем в json
result = {
    'user': MY_USER + 1,
    '1': newRates,
    '2': BEST_SUGGESTION
}
print(result)
import json
with open('result.json', 'w') as fp:
    json.dump(result, fp, indent=4)

{'user': 23, '1': {'Movie 3': 2.651, 'Movie 6': 3.474, 'Movie 7': 2.907, 'Movie 9': 3.144, 'Movie 21': 4.084, 'Movie 22': 3.144, 'Movie 26': 3.898}, '2': {'Movie 26': 4.145}}
